In [1]:
!curl https://storage.googleapis.com/wandb_datasets/nature_12K.zip --output nature_12K.zip
!unzip nature_12K.zip > /dev/null 2>&1
!rm nature_12K.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3639M  100 3639M    0     0  39.6M      0  0:01:31  0:01:31 --:--:-- 38.5M      0  0:01:34  0:00:39  0:00:55 40.1M 0     0  39.4M      0  0:01:32  0:01:13  0:00:19 39.6M
/bin/bash: line 1: rm nature_12K.zip: command not found


In [2]:
# WandB – Install the W&B library
%pip install wandb -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
# !wandb login

In [4]:
import wandb
wandb.login(key='130161b8988911058327a18dbbdfb663c58411b2')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m005 (da24m005-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
import wandb

### Question 1

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Set up data transformations to match ImageNet dimensions and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ImageNet dimensions
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load the dataset
train_dataset = datasets.ImageFolder('/kaggle/working/inaturalist_12K/train',
                                    transform=transform)
test_dataset = datasets.ImageFolder('/kaggle/working/inaturalist_12K/val',
                                   transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load a pre-trained model (ResNet50)
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer to match our 10 classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # 10 classes for iNaturalist


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 113MB/s]


### Question 2

In [7]:
def freeze_all_except_last(model):
    """Freeze all layers except the final classifier (feature extraction)"""
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the final layer
    for param in model.fc.parameters():
        param.requires_grad = True

    return model

def freeze_first_k_layers(model, k=7):
    """Freeze first k layer groups (partial fine-tuning)"""
    # Get all children
    children = list(model.children())

    # Freeze first k layer groups
    for i in range(k):
        for param in children[i].parameters():
            param.requires_grad = False

    return model

def progressive_unfreezing(model, current_epoch, unfreeze_epoch=5):
    """Progressively unfreeze layers as training progresses"""
    if current_epoch == 0:
        # Start with all layers frozen except the last
        for param in model.parameters():
            param.requires_grad = False
        for param in model.fc.parameters():
            param.requires_grad = True

    elif current_epoch == unfreeze_epoch:
        # Unfreeze the last convolutional block
        for param in model.layer4.parameters():
            param.requires_grad = True

    return model

### Question 3

In [8]:
def finetune_model(freeze_strategy="feature_extraction"):
    """Fine-tune the pre-trained model using the specified freezing strategy"""
    # Initialize wandb
    wandb.init(project="Assignment2_CNN_partB", name=f"resnet50-{freeze_strategy}")

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load pre-trained model
    model = models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)

    # Apply freezing strategy
    if freeze_strategy == "feature_extraction":
        model = freeze_all_except_last(model)
        print("Using feature extraction: all layers frozen except the last")
    elif freeze_strategy == "partial_finetuning":
        model = freeze_first_k_layers(model, k=7)
        print("Using partial fine-tuning: first 7 layer groups frozen")
    elif freeze_strategy == "progressive_unfreezing":
        model = progressive_unfreezing(model, current_epoch=0)
        print("Using progressive unfreezing: starting with all layers frozen except the last")

    model = model.to(device)

    # Count trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

    # Set up optimizer and loss function
    # Use a smaller learning rate for fine-tuning
    optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.0001)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    num_epochs = 25
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        # For progressive unfreezing, update which layers are frozen
        if freeze_strategy == "progressive_unfreezing":
            model = progressive_unfreezing(model, current_epoch=epoch)
            # Update optimizer to include newly unfrozen parameters
            if epoch == 5:  # Unfreeze epoch
                optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.00005)
                print("Unfreezing layer4, adjusting learning rate")

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = correct / total
        train_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        val_loss = val_loss / len(test_loader)

        # Log metrics
        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_accuracy": train_accuracy,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy,
            "trainable_parameters": trainable_params
        })

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}")

        # Save best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_finetuned_model.pth")
            print(f"New best model saved with accuracy: {best_accuracy:.2f}")

    # Final test evaluation
    model.load_state_dict(torch.load("best_finetuned_model.pth"))
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    final_accuracy = correct / total
    print(f"Final Test Accuracy: {final_accuracy:.2f}")
    wandb.log({"final_test_accuracy": final_accuracy})

    # Create a visualization of test predictions
    visualize_predictions(model, test_loader, device, train_dataset)

    wandb.finish()
    return model, final_accuracy

def visualize_predictions(model, test_loader, device, train_dataset):  # Add train_dataset as argument
    """Create a visualization of test predictions"""
    model.eval()
    images, labels = next(iter(test_loader))
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, preds = torch.max(outputs, 1)

    class_names = train_dataset.classes  # Get class names

    # Plot a grid of images with predictions
    fig, axes = plt.subplots(4, 8, figsize=(15, 8))
    axes = axes.flatten()

    for i in range(min(32, len(images))):
        img = images[i].cpu().permute(1, 2, 0).numpy()
        # Denormalize
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)

        axes[i].imshow(img)
        pred_class_name = class_names[preds[i].item()]  # Get predicted class name
        true_class_name = class_names[labels[i].item()]  # Get true class name
        color = 'green' if preds[i] == labels[i] else 'red'
        axes[i].set_title(f"Pred: {pred_class_name}", color=color)  # Display class name
        axes[i].axis('off')

    plt.tight_layout()
    wandb.log({"test_predictions": wandb.Image(fig)})
    plt.close()

# Run the fine-tuning with feature extraction strategy (freezing all layers except the last)
model, accuracy = finetune_model(freeze_strategy="feature_extraction")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: da24m005 (da24m005-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using feature extraction: all layers frozen except the last
Trainable parameters: 20,490 (0.09% of total)
Epoch 1/25, Train Loss: 1.7203, Train Acc: 0.53%, Val Loss: 1.2960, Val Acc: 0.67%
New best model saved with accuracy: 0.67%
Epoch 2/25, Train Loss: 1.1799, Train Acc: 0.68%, Val Loss: 1.0195, Val Acc: 0.71%
New best model saved with accuracy: 0.71%
Epoch 3/25, Train Loss: 1.0097, Train Acc: 0.71%, Val Loss: 0.9191, Val Acc: 0.73%
New best model saved with accuracy: 0.73%
Epoch 4/25, Train Loss: 0.9232, Train Acc: 0.73%, Val Loss: 0.8466, Val Acc: 0.74%
New best model saved with accuracy: 0.74%
Epoch 5/25, Train Loss: 0.8773, Train Acc: 0.73%, Val Loss: 0.8138, Val Acc: 0.75%
New best model saved with accuracy: 0.75%
Epoch 6/25, Train Loss: 0.8424, Train Acc: 0.74%, Val Loss: 0.7890, Val Acc: 0.75%
New best model saved with accuracy: 0.75%
Epoch 7/25, Train Loss: 0.8173, Train Acc: 0.75%, Val Loss: 0.7663, Val Acc: 0.76%
New best model saved with accuracy: 0.76%
Epoch 8/25, Train L

TypeError: visualize_predictions() missing 1 required positional argument: 'train_dataset'